In [6]:
scope.dis()
target.dis()

In [7]:
exit()

In [1]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
scope.adc.samples=24400

In [6]:
# load(scope, "masked")

scope.adc.samples=100
# tc = scope.adc.trig_count
x_ = [random.randrange(2) for _ in range(16)]
y_ = [random.randrange(2) for _ in range(16)]
x_t = [random.randrange(2) for _ in range(16)]
y_t = [random.randrange(2) for _ in range(16)]
msg = xor(x_,x_t) + x_t + xor(y_,y_t) + y_t + [0 for _ in range(64)]
target.simpleserial_write('p', msg)
samps = scope.adc.trig_count
print(samps)

11906686


In [16]:
# masking inside component
# function
def setup():
    load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand, current_puf
    ["set1", "set2", "set1_3", "set1_3"]
    if op == "set1":
        p_temp = set_1
    elif op == "set2":
        p_temp = set_2
    elif op == "set1_3":
        p_temp = set_1d3
    elif op == "set1_3":
        p_temp = set_2d3
    else:
        raise "a"

    
    posture(target,'k', get_bytes(p_temp), 20)
    # p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
    current_puf = p_temp
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (get_bits(fdb)[:64] != current_puf[-64:]):
        print("WRONG!")
        print(xor(get_bits(fdb)[:64],current_puf[-64:]))
        print()
        return True
    pass


# k = [random.randrange(2) for _ in range(64)]
set_1 = [random.randrange(2) for _ in range(127)]
word = s.encode([random.randrange(2) for _ in range(64)])
set_2 = xor(set_1, word)

set_1d3 = list(set_1)
for i in random.choices(range(127),k=3):
    set_1d3[i] ^= 1
    
set_2d3 = list(set_2)
for i in random.choices(range(127),k=3):
    set_2d3[i] ^= 1
    
rand = None
current_puf = None
prefix_name = "constant_HD"

env = ENV()
# operation settings
env.root = "C:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["set1", "set2", "set1_3", "set2_3"] # operations list , "set2", "set3"
env.subfolder = "syndrome" # subfolder name
env.TRACES_NUM = 2000 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = "The memory masking is done before trigger"
env.title = ""
env.name = prefix_name
env.readme = """masked decoding with syndrome scheme
set vs set PUF experiments: set vs set+1,and+127/2 different bits
masking is done inside component, but before trigger.
"""
s = Scheme() # scheme execution helper

# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(3)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)

env.title = "PUF set vs set with 1 FI, constant key"
env.name = prefix_name + "_d1"
split_file(env, file_names, [0, 1], "set1")


env.title = "PUF set vs set, constant key"
env.name = prefix_name + "_rand"
split_file(env, file_names, [0, 2], "set_half")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13515 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13515 bytes
None
file0 file1 file2 done 2024-07-09 12-33


In [16]:
# masking inside component
# function
def setup():
    load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand, current_puf
    if op == "set":
        p_temp = set_1
    elif op == "set1":
        p_temp = set_c1
    elif op == "set2":
        p_temp = set_c2
    elif op == "set3":
        p_temp = set_c3
    elif op == "set_half":
        p_temp = set_2
    else:
        raise "a"

    
    posture(target,'k', get_bytes(p_temp), 20)
    # p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
    current_puf = p_temp
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (get_bits(fdb)[:64] != current_puf[-64:]):
        print("WRONG!")
        print(xor(get_bits(fdb)[:64],current_puf[-64:]))
        print()
        return True
    pass


# k = [random.randrange(2) for _ in range(64)]
set_1 = [random.randrange(2) for _ in range(127)]
set_2 = [random.randrange(2) for _ in range(127)]
set_c1 = list(set_1)
for i in random.choices(range(127),k=1):
    set_c1[i] ^= 1
set_c2 = list(set_1)
for i in random.choices(range(127),k=2):
    set_c2[i] ^= 1
set_c3 = list(set_1)
for i in random.choices(range(127),k=3):
    set_c3[i] ^= 1
rand = None
current_puf = None
prefix_name = "set_vs_set_out"

env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["set", "set1", "set_half"] # operations list , "set2", "set3"
env.subfolder = "syndrome" # subfolder name
env.TRACES_NUM = 2000 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = "The memory masking is done before trigger"
env.title = ""
env.name = prefix_name
env.readme = """masked decoding with syndrome scheme
set vs set PUF experiments: set vs set+1,and+127/2 different bits
masking is done inside component, but before trigger.
"""
s = Scheme() # scheme execution helper

# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(3)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)

env.title = "PUF set vs set with 1 FI, constant key"
env.name = prefix_name + "_d1"
split_file(env, file_names, [0, 1], "set1")


env.title = "PUF set vs set, constant key"
env.name = prefix_name + "_rand"
split_file(env, file_names, [0, 2], "set_half")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13515 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13515 bytes
None
file0 file1 file2 done 2024-07-09 12-33


In [4]:
# masking outside component
# function
def setup():
    print(start_time)
    # load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    # load(scope, "../c-files/testsyn-CWLITEARM")
    pass
    
def execute(index, op, t):
    global rand
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand, current_puf
    if op == "set":
        p_temp = set_1
    elif op == "set1":
        p_temp = set_c1
    elif op == "set2":
        p_temp = set_c2
    elif op == "set3":
        p_temp = set_c3
    elif op == "set_half":
        p_temp = set_2
    else:
        raise "a"

    
    posture(target,'k', get_bytes(p_temp), 20)
    rand = [random.randrange(2) for _ in range(64)]
    # p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
    current_puf = p_temp
    return get_bytes(xor(p_temp, s.encode(rand)))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if fdb[15] != 0:
        print("NOOOOOOOOOOOO", fdb[15])
    if (xor(rand, get_bits(fdb)[:64]) != current_puf[-64:]):
        print("WRONG!")
        print(fdb)
        print(xor(rand, get_bits(fdb)[:64],current_puf[-64:]))
        print()
        return True
    pass


# k = [random.randrange(2) for _ in range(64)]
set_1 = [random.randrange(2) for _ in range(127)]
set_2 = [random.randrange(2) for _ in range(127)]
set_c1 = list(set_1)
for i in random.choices(range(127),k=1):
    set_c1[i] ^= 1
set_c2 = list(set_1)
for i in random.choices(range(127),k=2):
    set_c2[i] ^= 1
set_c3 = list(set_1)
for i in random.choices(range(127),k=3):
    set_c3[i] ^= 1
rand = None
current_puf = None
prefix_name = "set_vs_set_outreal2"

env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["set", "set1", "set_half"] # operations list , "set2", "set3"
env.subfolder = "syndrome" # subfolder name
env.TRACES_NUM = 2000 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = "Masking done outside component"
env.title = ""
env.name = prefix_name
env.readme = """masked decoding with syndrome scheme
set vs set PUF experiments: set vs set+1,and+127/2 different bits
masking is done outside component. No visible info on PUF in component besides error.
Scheme is a bit different - we recieve back the error, and XOR it with the masked word.
after that, we XOR the masked word back into the normal one outside the component
"""
s = Scheme() # scheme execution helper

# run loop
start_time = date_str()
print(setup())
time.sleep(0.1)


file_names = [f"file{i}" for i in range(3)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)

env.title = "PUF set vs set with 1 FI, constant key"
env.name = prefix_name + "_d1"
split_file(env, file_names, [0, 1], "set1")


env.title = "PUF set vs set, constant key"
env.name = prefix_name + "_rand"
split_file(env, file_names, [0, 2], "set_half")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


2024-07-11 22-50
None
file0 file1 file2 done 2024-07-11 23-15


In [13]:
set_1 = [random.randrange(1) for _ in range(127)]
print(set_1)
print(s.syndrome(set_1))
a = posture(target,'k', get_bytes(set_1), 20)
print(a)
b = posture(target, 'p', get_bytes(set_1), 16)
print(b)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
bytearray(b'_30\x1b\x14\x1e{C\x03\x164R\x1b;ye\x15\x05z\x12')
bytearray(b'\x00\x00\x10\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


In [13]:

# function
def setup():
    load(scope,"masked-inside")
    # load(scope, "../c-files/masked-decode")
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op == "set":
        p_temp = set_1
    elif op == "set1":
        p_temp = set_c1
    elif op == "set2":
        p_temp = set_c2
    elif op == "set3":
        p_temp = set_c3
    elif op == "set_half":
        p_temp = set_2
    else:
        raise "a"
    rand1 = [random.randrange(2) for _ in range(64)]
    rand = [random.randrange(1) for _ in range(64)]
    mem = xor(p_temp, s.encode(k))
    posture(target,'k', get_bytes(mem), 16)
    rand = [random.randrange(2) for _ in range(64)]
    p_temp = p_temp # , mem, s.encode(rand)
    rand = xor(rand,rand1)
    x = get_bytes(p_temp + [0] + s.encode(rand))
    return x

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (xor(get_bits(fdb)[:64], rand) != k):
        print("WRONG!")
        return True
    pass


k = [random.randrange(2) for _ in range(64)]
set_1 = [random.randrange(2) for _ in range(127)]
set_2 = [random.randrange(2) for _ in range(127)]
set_c1 = list(set_1)
for i in random.choices(range(127),k=1):
    set_c1[i] ^= 1
set_c2 = list(set_1)
for i in random.choices(range(127),k=2):
    set_c2[i] ^= 1
set_c3 = list(set_1)
for i in random.choices(range(127),k=3):
    set_c3[i] ^= 1
rand = None
prefix_name = "set_vs_set_in"

env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["set", "set1", "set2", "set3", "set_half"] # operations list
env.subfolder = "masked2" # subfolder name
env.TRACES_NUM = 2000 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masked decoding
constant key
set vs set PUF experiments: set vs set+1,+2,+3,and+127/2 different bits
masking is done on key exclusively inside the component.
"""
s = Scheme() # scheme execution helper

# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()


file_names = [f"file{i}" for i in range(3)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)


env.title = "PUF set vs set with 1 FI, constant key"
env.name = prefix_name + "_d1"
split_file(env, file_names, [0, 1], "set1")

env.title = "PUF set vs set with 2 FIs, constant key"
env.name = prefix_name + "_d2"
split_file(env, file_names, [0, 2], "set2")

env.title = "PUF set vs set with 3 FIs, constant key"
env.name = prefix_name + "_d3"
split_file(env, file_names, [0, 3], "set3")

env.title = "PUF set vs set, constant key"
env.name = prefix_name + "_rand"
split_file(env, file_names, [0, 4], "set_half")

# split_file(["file_name"], [0, 1], "result")
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13127 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13127 bytes
None
file0 file1 file2 done 2024-07-09 10-43


In [ ]:
--------------------

In [11]:
# masked
# function
def setup():
    # runs once in the start -> prints returned
    #load("masked")
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op in ["random", "tvla"]:
        p_temp = [random.randrange(2) for _ in range(127)]
    elif op == "set":
        p_temp = p1.puf
    elif op == "zero":
        p_temp = p0.puf
    else:
        return None
    rand1 = [random.randrange(2) for _ in range(64)]
    rand = [random.randrange(2) for _ in range(64)]
    mem = xor(p_temp, s.encode(k))
    posture(target, 'k', get_bytes(xor(mem, s.encode(rand1))), 16)
    rand = [random.randrange(2) for _ in range(64)]
    p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
    rand = xor(rand,rand1)
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> True <=> trace isnt taken
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (xor(get_bits(fdb)[:64], rand) != k):
        print("WRONG!")
        return True
    pass

# operation settings
root = "F:/programs/traces"
operations = ["set", "set1", "set2", "set16"] # operations list
subfolder = "masked" # subfolder name
TRACES_NUM = 1000 # number of traces
scope.adc.samples = 20000 # number of samples per trace
simulate = False # doesnt do anything for now. should be False
s = Scheme() # scheme execution helper

rand = None

p0 = PUF([0 for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
p1 = PUF([random.randrange(2) for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
k = [random.randrange(2) for _ in range(64)]
set_weight = sum(p1.puf)
key_weight = sum(k)

print(setup())
time.sleep(0.1)
start_time = date_str()
##
file_names = [f"file{i}" for i in range(3)]
for file_name in file_names:
    status = traces(file_name, 0)
split_file(file_names, [0, 1], "set_diff1")
split_file(file_names, [0, 2], "set_diff2")
split_file(file_names, [0, 3], "set_diff3")
split_file(file_names, [0, 4], "set_rand")
remove_files(file_names)
##
generate_file("start_time", "operations", "set_weight", "key_weight")
print("done")

None


TypeError: traces() missing 2 required positional arguments: 'target' and 'env'

In [ ]:
# masked
# function
def setup():
    # runs once in the start -> prints returned
    #load("masked")
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op in ["random", "tvla"]:
        p_temp = [random.randrange(2) for _ in range(127)]
    elif op == "set":
        p_temp = p1.puf
    elif op == "zero":
        p_temp = p0.puf
    else:
        return None
    rand1 = [random.randrange(2) for _ in range(64)]
    rand = [random.randrange(2) for _ in range(64)]
    mem = xor(p_temp, s.encode(k))
    posture('k', get_bytes(xor(mem, s.encode(rand1))), 16)
    rand = [random.randrange(2) for _ in range(64)]
    p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
    rand = xor(rand,rand1)
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> True <=> trace isnt taken
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (xor(get_bits(fdb)[:64], rand) != k):
        print("WRONG!")
        return True
    pass

# operation settings
root = "F:/programs/traces"
operations = ["set", "set1", "set2", "set16"] # operations list
subfolder = "masked" # subfolder name
TRACES_NUM = 1000 # number of traces
scope.adc.samples = 20000 # number of samples per trace
simulate = False # doesnt do anything for now. should be False
s = Scheme() # scheme execution helper

rand = None

p0 = PUF([0 for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
p1 = PUF([random.randrange(2) for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
k = [random.randrange(2) for _ in range(64)]
set_weight = sum(p1.puf)
key_weight = sum(k)

print(setup())
time.sleep(0.1)
start_time = date_str()
##
file_names = [f"file{i}" for i in range(3)]
for file_name in file_names:
    status = traces(file_name, 0)
split_file(file_names, [0, 1], "set_diff1")
split_file(file_names, [0, 2], "set_diff2")
split_file(file_names, [0, 3], "set_diff3")
split_file(file_names, [0, 4], "set_rand")
remove_files(file_names)
##
generate_file("start_time", "operations", "set_weight", "key_weight")
print("done")

In [2]:
# masked random PUF
# function
def setup():
    # runs once in the start -> prints returned
    load(scope, "../c-files/masked-CWLITEARM")
    print(date_str())
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op in ["random", "tvla"]:
        p_temp = [random.randrange(2) for _ in range(127)]
    elif op == "set":
        p_temp = p1.puf
    elif op == "zero":
        p_temp = p0.puf
    else:
        return None
    rand1 = [random.randrange(1) for _ in range(64)]
    rand = [random.randrange(2) for _ in range(64)]
    mem = xor(p_temp, s.encode(k))
    posture(target,'k', get_bytes(xor(mem, s.encode(rand1))), 16)
    p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
    rand = xor(rand,rand1)
    # time.sleep(0.005)
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> True <=> trace isnt taken
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (xor(get_bits(fdb)[:64], rand) != k):
        print("WRONG!")
        return True
    pass


env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["random", "tvla", "set", "zero"] # operations list
env.subfolder = "masked" # subfolder name
env.TRACES_NUM = 1250 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.title = ""
env.desc = "helper data's mask is not refreshed"
prefix_name = "omasked_decode"
env.name = prefix_name
env.readme = """masked decoding
TVLA between set key + puf against set key + randomized PUF
additional randomized puf vs randomized puf for sanity
only puf is masked. memory is exposed"""

s = Scheme() # scheme execution helper


rand = None

p0 = PUF([0 for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
p1 = PUF([random.randrange(2) for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
k = [random.randrange(2) for _ in range(64)]
set_weight = sum(p1.puf)
key_weight = sum(k)


# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(2)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    


env.title = "One masked Decode TVLA random vs random"
env.name = prefix_name + "_rand"
split_file(env, file_names, [0, 1], "rand")

env.title = "One masked Decode TVLA random vs set"
env.name = prefix_name + "_set"
split_file(env, file_names, [0, 2], "set")

env.title = "One masked Decode TVLA random vs zero"
env.name = prefix_name + "_zero"
split_file(env, file_names, [0, 3], "zero")


remove_files(env, file_names)

##

generate_file(env, globals(), "start_time", "operations", "set_weight", "key_weight")
print("done", date_str())


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13107 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13107 bytes
2024-06-14 09-40
None
file0 file1 done 2024-06-14 09-54


In [5]:
# masked random KEY
# function
def setup():
    # runs once in the start -> prints returned
    # load(scope, "../c-files/masked-CWLITEARM")
    print(date_str())
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op in ["random", "tvla"]:
        k_temp = [random.randrange(2) for _ in range(64)]
    elif op == "set":
        k_temp = k1
    elif op == "zero":
        k_temp = k0
    else:
        return None
    rand1 = [random.randrange(2) for _ in range(64)]
    rand = [random.randrange(1) for _ in range(64)]
    mem = xor(puf, s.encode(k))
    posture(target,'k', get_bytes(xor(mem, s.encode(rand1))), 16)
    p_temp = xor(puf, s.encode(rand)) # , mem, s.encode(rand)
    rand = xor(rand,rand1)
    # time.sleep(0.005)
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> True <=> trace isnt taken
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (xor(get_bits(fdb)[:64], rand) != k):
        print("WRONG!")
        return True
    pass


env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["random", "tvla", "set", "zero"] # operations list
env.subfolder = "masked" # subfolder name
env.TRACES_NUM = 1250 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.title = ""

env.desc = "PUF's mask is not refreshed"
prefix_name = "okey_masked_decode"
env.name = prefix_name
env.readme = """masked decoding
TVLA between set key + puf against random key + set PUF
additional randomized key vs randomized key for sanity
public memory is masked, but PUF isnt"""

s = Scheme() # scheme execution helper


rand = None

k0 = [0 for _ in range(64)] #PUF([random.randrange(2) for _ in range(127)])
k1 = [random.randrange(2) for _ in range(64)] #PUF([random.randrange(2) for _ in range(127)])
puf = [random.randrange(2) for _ in range(127)]
set_weight = sum(p1.puf)
key_weight = sum(k)


# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(2)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    


env.title = "Once key masked Decode TVLA random vs random"
env.name = prefix_name + "_rand"
split_file(env, file_names, [0, 1], "rand")

env.title = "Once Key masked Decode TVLA random vs set"
env.name = prefix_name + "_set"
split_file(env, file_names, [0, 2], "set")

env.title = "Once Key masked Decode TVLA random vs zero"
env.name = prefix_name + "_zero"
split_file(env, file_names, [0, 3], "zero")


remove_files(env, file_names)

##

generate_file(env, globals(), "start_time", "operations", "set_weight", "key_weight")
print("done", date_str())


2024-06-14 10-24
None
file0 file1 done 2024-06-14 10-37


In [5]:
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op in ["random", "tvla"]:
        p_temp = [random.randrange(2) for _ in range(127)]
    elif op == "set":
        p_temp = p1.puf
    elif op == "zero":
        p_temp = p0.puf
    else:
        return None
    rand1 = [random.randrange(1) for _ in range(64)]
    rand = [random.randrange(2) for _ in range(64)]
    mem = xor(p_temp, s.encode(k))
    x = get_bytes(xor(mem, s.encode(rand1)))
    print(x, len(x))
    # posture(target,'k', get_bytes(xor(mem, s.encode(rand1))), 16)
    print("here1")
    target.simpleserial_write('k', x)
    time.sleep(0.1)
    target.simpleserial_read('r', 16)
    print("here2")
    # target.flush()
    p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
    rand = xor(rand,rand1)
    return get_bytes(p_temp)

# load(scope, "../c-files/masked-CWLITEARM")
target.flush()
msg = execute(0, "random", 0)
time.sleep(0.01)
# target.flush()
scope.arm()
print(msg, len(msg))
target.simpleserial_write('p', msg)
ret = scope.capture()
if ret:
    print("ERROR")
traces = scope.get_last_trace()
if env.read > 0:
    fdb = target.simpleserial_read('r', env.read)
print(1234)

bytearray(b'\xd1\xb90-\x85\x8e*\xaf\x9b\xa4\x93\xa4\xbd@$|') 16
here1


(ChipWhisperer Target WARNING|File SimpleSerial.py:410) Unexpected start to command: 


here2
bytearray(b'\xe3\xba"\x94\x807\xcbSZe\x1d,\x19\xe8\x86i') 16


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a


ERROR


(ChipWhisperer Target WARNING|File SimpleSerial.py:410) Unexpected start to command: 


1234


In [4]:


rand = [random.randrange(2) for _ in range(64)]
s = Scheme() 
# p_temp = xor(p_temp, s.encode(rand)) # , mem, s.encode(rand)
print(rand)
a = get_bytes(s.encode(rand))

print(a)
b= s.encode(rand)
print(len(b),b)
print(s.decode(get_bits(a)[:127]))

[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0]
hehehe
bytearray(b'?\xa6\x9a@\xa1\xa9\xe3U\xdcm\x1c=\xfe\x10\xd5\x1a')
127 [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0]
hehehe
hehehe
[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0]


In [3]:
print(get_bytes([1,0,0,0,0,0,0,0,1]))
print(get_bytes([1,0,0,0]))

hehehe
bytearray(b'\x01\x01')
hehehe
bytearray(b'\x01')
